1. (전체)pdf 파일을 csv로 만들기

In [1]:
# ## -> develop 및 검증 필요 누락된 내용 발견

# import pdfplumber
# import pandas as pd
# import re

# # PDF 파일 경로
# pdf_file_path = r"./libs/pdf/3-1. 강의시간표 전체(2024-2).pdf"

# # 저장할 CSV 파일 경로 .\libs\csv
# csv_file_path = ".\libs\csv/kwu-24-2-timetable.csv"

# # 데이터를 저장할 리스트 초기화
# data = []

# # PDF 파일 열기
# with pdfplumber.open(pdf_file_path) as pdf:
#     # 각 페이지 순회
#     for page in pdf.pages:
#         # 페이지에서 표 추출
#         table = page.extract_table()
#         if table:
#             # 추출된 표 데이터를 리스트에 추가
#             data.extend(table)

# # 데이터가 없는 빈 행을 필터링
# filtered_data = [row for row in data if any(cell.strip() for cell in row)]

# # 추출된 데이터를 DataFrame으로 변환
# df = pd.DataFrame(filtered_data[1:], columns=filtered_data[0])  # 첫 번째 행을 헤더로 사용

# # DataFrame을 CSV 파일로 저장
# df.to_csv(csv_file_path, index=False)

# print("CSV 파일이 저장되었습니다.")


# # 저장할 CSV 파일 경로 .\libs\csv
# # csv_file_path = ".\libs\csv/kwu-24-2-timetable.csv"

In [2]:
# 1. pdf -> excel -> csv
import pandas as pd

df = pd.read_csv(r'./libs/csv/누락된교양.csv', encoding='cp949')

2. 해당 csv 파일에서 학정번호형식을 제외한 '비고', '헤더피쳐'가 중복되고 필요없는 정보임으로 삭제 

In [3]:
import re

# 학정번호 형식을 정규 표현식으로 정의 (예: I040-2-6387-02)
pattern = r'^[A-Za-z0-9]+-\d-\d{4}-\d{2}$'

# 학정번호 형식과 일치하는 행만 남기기
df = df[df['학정번호'].apply(lambda x: bool(re.match(pattern, str(x))))]

In [4]:
df

,학정번호,과목명,분반,연계,이수,학점,시수,담당교수,"강의시간 (요일,교시)",강의유형
0,0000-1-3093-01,한국영화사로보는가족,NaN,NaN,교선,3,3,강성률,"금7,8",원격수업100?
1,0000-1-4428-01,디지털신기술기반신재생에너지,NaN,NaN,교선,3,3,손희상,"화7,8",원격수업100?
2,0000-1-4430-01,"슬기로운가정생활,관계놀이",NaN,NaN,교선,3,3,최윤희 윤란호,"목8,9","원격수업100,팀티칭강의"
3,0000-1-4430-02,"슬기로운가정생활,관계놀이",외국인만 수강가능,NaN,교선,3,3,최윤희 윤란호,"목8,9","원격수업101,팀티칭강의,외국인전 용"
4,0000-1-4431-01,ICT가바꾸는미래의법,NaN,NaN,교선,3,3,선지원,"화5,6",원격수업100?
5,0000-1-4848-01,현대중국예술과문화코드이해,NaN,NaN,교선,3,3,전보옥,"월7,8",원격수업100?
6,0000-1-4849-01,배터리신기술과우리의생활,NaN,NaN,교선,3,3,손희상,"목7,8",원격수업100?
7,0000-1-4850-01,수학적사고로보는세상,NaN,NaN,교선,3,3,최종성,"수7,8",원격수업100?
8,0000-3-9747-01,안전및조직관리사례연구,NaN,NaN,무관,3,4,NaN,"화1,2,목1,2",NaN
9,0000-4-9748-01,조직리더십사례연구,NaN,NaN,무관,3,4,NaN,"화1,2,목1,2",NaN


3. '학정번호' 칼럼을 기준으로 중복값 출력 및 처리 (매치업 강의는 전선을 남기고 교선을 없애기)

In [5]:
# 학정번호별 중복 횟수 계산
duplicates_count = df['학정번호'].value_counts()

# 2회 이상 중복된 학정번호만 선택
duplicates_count = duplicates_count[duplicates_count > 1]

# 중복 횟수 내림차순으로 정렬
duplicates_count = duplicates_count.sort_values(ascending=False)

print("학정번호별 중복 횟수:")
print(duplicates_count)

학정번호별 중복 횟수:
Series([], Name: count, dtype: int64)


In [6]:
# 중복된 학정번호 중 하나의 행만 남기고 나머지 삭제
df_unique = df.drop_duplicates(subset=['학정번호'], keep='first')

In [7]:
# 학정번호별 중복 횟수 계산
duplicates_count = df_unique['학정번호'].value_counts()

# 2회 이상 중복된 학정번호만 선택
duplicates_count = duplicates_count[duplicates_count > 1]

# 중복 횟수 내림차순으로 정렬
duplicates_count = duplicates_count.sort_values(ascending=False)

print("학정번호별 중복 횟수:")
print(duplicates_count)

학정번호별 중복 횟수:
Series([], Name: count, dtype: int64)


In [8]:
df_unique.shape

(11, 10)

In [9]:
df = df_unique 

In [10]:
df.shape

(11, 10)

### 4. 분반에 대한 단어 포함관계 확인

분반에서 추출할 수 있는 정보는 
1. 학과별 분반 정보 ex)"공대합반", "인융대(A1,A2)", "V1,V4", "J1~J3", 참빛인재대학 전용, '타학과생(다전공생 포함)\n수강불가'
2. 외국인 ex)외국인만 수강가능 , 외국인 수강 불가
3. 학년정보 1학년 2학년 3학년
4. 체육특기자만 수강 가능 여부
5. 재수강용

In [11]:
import pandas as pd

# CSV 파일 읽어오기
# df = pd.read_csv('강의_전체.csv', encoding='euc-kr')

# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('체육특기', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
[]


In [12]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('외국인', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['외국인만 수강가능']


In [13]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('참빛', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
[]


In [14]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('타학과', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
[]


In [15]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('학년', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
[]


In [16]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('재수강', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)

분반에 포함된 유니크한 값:
[]


In [17]:
import pandas as pd
import re

# CSV 파일 읽어오기
# df = pd.read_csv('강의_전체.csv', encoding='euc-kr')

# NaN 값을 빈 문자열로 대체
df['분반'] = df['분반'].fillna("")
# NaN 값을 빈 문자열로 대체
df['분반_원본'] = df['분반'].fillna("")


In [18]:
# 조건에 따라 열 값 수정
conditions_foreign = ['외국인만 수강가능', '공대 합반\n외국인만수강가능', '전정대 합반 외국인만 수강가능']
condition_domestic = '외국인학생 수강불가'


# 외국인재학생 = True, 국내재학생 = False 설정
for condition in conditions_foreign:
    df.loc[df['분반'] == condition, ['외국인재학생', '국내재학생']] = [True, False]


# 외국인재학생 = False, 국내재학생 = True 설정
df.loc[df['분반'] == condition_domestic, ['외국인재학생', '국내재학생']] = [False, True]

## 5. 칼럼 추가 

In [19]:
# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'V1,V2,V3,V4' 할당
df['분반_학과1'] = df['분반'].apply(lambda x: 'V1,V2,V3,V4' if '참빛인재대학' in x else "")

# '체육특기자'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['체육특기자'] = df['분반'].apply(lambda x: 'TRUE' if '체육특기자' in x else "")

# '재수강용'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['재수강용'] = df['분반'].apply(lambda x: 'TRUE' if '재수강' in x else "")

# '타학과수강불가'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['타학과수강불가'] = df['분반'].apply(lambda x: 'FALSE' if '타학과' in x else "")

In [20]:
# 삭제할 문자열 리스트
remove_items = ['체육실기', '음악실기', '미술실기']

# 특정 문자열 삭제
for item in remove_items:
    df['분반'] = df['분반'].str.replace(item, '')


In [21]:
df.columns

Index(['학정번호', '과목명', '분반', '연계', '이수', '학점', '시수', '담당교수', '강의시간 (요일,교시)',
       '강의유형', '분반_원본', '외국인재학생', '국내재학생', '분반_학과1', '체육특기자', '재수강용',
       '타학과수강불가'],
      dtype='object')

## 6. 학년 처리

In [22]:
# 새로운 열 추가를 위한 함수 정의
def extract_grade_info(row):
    grades = []
    if '1학년' in row:
        grades.append('1')
    if '2학년' in row:
        grades.append('2')
    if '3학년' in row:
        grades.append('3')
    if '2~4학년' in row:
        grades.append('2,3,4')
    return ','.join(grades)

# 새로운 열 '학년 정보' 추가
df['학년'] = df['분반'].apply(extract_grade_info)
df['학년'].value_counts()

학년
    11
Name: count, dtype: int64

## 7. 분반 처리

In [23]:

# NaN 값을 빈 문자열로 대체
df['분반'] = df['분반'].fillna("")

#A1~A2이런식으로 생긴 애들 모음
def expand_ranges(text):
    # 정규 표현식을 사용하여 패턴을 인식하고 대체
    pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')

    # 각 패턴에 대해 확장된 범위로 대체하고, 그 결과만 반환
    def replace_range(match):
        start_char = match.group(1)
        start_num = int(match.group(2))
        end_num = int(match.group(3))
        return ','.join(f"{start_char}{i}" for i in range(start_num, end_num + 1))
    
    # 전체 문자열 대신, 일치하는 부분만 찾아서 리스트로 모아 반환
    expanded_parts = pattern.findall(text)
    expanded_results = [replace_range(match) for match in pattern.finditer(text)]
    
    return ','.join(expanded_results)  # 리스트를 문자열로 결합
def replace_spaces_with_commas(text):
    # 모든 공백을 콤마로 대체
    return text.replace(" ", ",")

# '분반' 열에 함수 적용
df['분반_학과'] = df['분반'].apply(expand_ranges)

# 결과 출력
print(df)

              학정번호             과목명         분반  연계  이수  학점  시수     담당교수  \
0   0000-1-3093-01      한국영화사로보는가족            NaN  교선   3   3      강성률   
1   0000-1-4428-01  디지털신기술기반신재생에너지            NaN  교선   3   3      손희상   
2   0000-1-4430-01   슬기로운가정생활,관계놀이            NaN  교선   3   3  최윤희 윤란호   
3   0000-1-4430-02   슬기로운가정생활,관계놀이  외국인만 수강가능 NaN  교선   3   3  최윤희 윤란호   
4   0000-1-4431-01     ICT가바꾸는미래의법            NaN  교선   3   3      선지원   
5   0000-1-4848-01   현대중국예술과문화코드이해            NaN  교선   3   3      전보옥   
6   0000-1-4849-01    배터리신기술과우리의생활            NaN  교선   3   3      손희상   
7   0000-1-4850-01      수학적사고로보는세상            NaN  교선   3   3      최종성   
8   0000-3-9747-01     안전및조직관리사례연구            NaN  무관   3   4      NaN   
9   0000-4-9748-01       조직리더십사례연구            NaN  무관   3   4      NaN   
10  0000-1-3675-01    외국인학생을위한기초수학  외국인만 수강가능 NaN  교선   3   3      최현옥   

   강의시간 (요일,교시)                  강의유형      분반_원본 외국인재학생  국내재학생 분반_학과1 체육특기자  \
0          금7,8              원격수

In [24]:
def remove_range_patterns(text):
    # 정규 표현식을 사용하여 패턴을 인식하고 삭제
    pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')
    
    # 패턴과 일치하는 부분을 빈 문자열로 대체
    result = pattern.sub('', str(text))
    
    # 결과 문자열의 앞뒤 공백 제거
    return result.strip()

# '분반' 열에 함수 적용
df['분반'] = df['분반'].apply(remove_range_patterns)

In [25]:
#A1,A2이런식으로 생긴 애들 모음

def separate_alphanumeric_patterns(text):
    # 알파벳 하나와 숫자로 이루어진 패턴을 찾는 정규 표현식
    pattern = re.compile(r'([A-Z]\d+)')
    
    # 패턴과 일치하는 모든 부분 찾기
    matches = pattern.findall(str(text))
    
    # 패턴과 일치하는 부분을 제거한 나머지 텍스트
    remaining_text = pattern.sub('', str(text)).strip()
    
    # 찾은 패턴들을 콤마로 구분하여 하나의 문자열로 만듦
    patterns = ','.join(matches) if matches else ''
    
    return remaining_text, patterns


# '분반' 열에 함수 적용
df['분반_정리'], df['분반_패턴'] = zip(*df['분반'].apply(separate_alphanumeric_patterns))

# 결과 출력
print(df)

              학정번호             과목명         분반  연계  이수  학점  시수     담당교수  \
0   0000-1-3093-01      한국영화사로보는가족            NaN  교선   3   3      강성률   
1   0000-1-4428-01  디지털신기술기반신재생에너지            NaN  교선   3   3      손희상   
2   0000-1-4430-01   슬기로운가정생활,관계놀이            NaN  교선   3   3  최윤희 윤란호   
3   0000-1-4430-02   슬기로운가정생활,관계놀이  외국인만 수강가능 NaN  교선   3   3  최윤희 윤란호   
4   0000-1-4431-01     ICT가바꾸는미래의법            NaN  교선   3   3      선지원   
5   0000-1-4848-01   현대중국예술과문화코드이해            NaN  교선   3   3      전보옥   
6   0000-1-4849-01    배터리신기술과우리의생활            NaN  교선   3   3      손희상   
7   0000-1-4850-01      수학적사고로보는세상            NaN  교선   3   3      최종성   
8   0000-3-9747-01     안전및조직관리사례연구            NaN  무관   3   4      NaN   
9   0000-4-9748-01       조직리더십사례연구            NaN  무관   3   4      NaN   
10  0000-1-3675-01    외국인학생을위한기초수학  외국인만 수강가능 NaN  교선   3   3      최현옥   

   강의시간 (요일,교시)                  강의유형  ... 외국인재학생  국내재학생 분반_학과1 체육특기자 재수강용  \
0          금7,8              원격수업

In [26]:
def combine_and_deduplicate(row):
    # '분반_학과'와 '분반_패턴'의 값을 합칩니다
    combined = f"{row['분반_학과']},{row['분반_학과1']},{row['분반_패턴']}"
    
    # 콤마로 분리하고 공백을 제거합니다
    patterns = [p.strip() for p in combined.split(',') if p.strip()]
    
    # 중복을 제거하고 정렬합니다
    unique_patterns = sorted(set(patterns))
    
    # 다시 콤마로 구분된 문자열로 만듭니다
    return ','.join(unique_patterns)

# '분반_최종' 열 생성
df['분반_최종'] = df.apply(combine_and_deduplicate, axis=1)

# 결과 출력
print(df[['분반_학과', '분반_패턴','분반_학과1', '분반_최종']])

# 불필요한 열 삭제
columns_to_drop = ['분반_학과', '분반_학과1', '분반_정리', '분반_패턴', '분반']
df = df.drop(columns=columns_to_drop)

   분반_학과 분반_패턴 분반_학과1 분반_최종
0                          
1                          
2                          
3                          
4                          
5                          
6                          
7                          
8                          
9                          
10                         


In [27]:
#타학과생(다전공생 포함)\n수강불가 -> 수동으로 분반_최종에 추가하는 처리 필요

## 추가로 해야할 일

<2학기>

1. 전공인정분반 추가 -> 윤나
2. 타학과전공인정과목분반 추가
3. 선수과목 관계 추가

여기서 추가로

1. 참빛인재 칼럼 추가 -> 해결 
2. 강의시간 좌표형 변환 -> 윤나
3. 강의 테마 칼럼 추가 -> 윤나

<윤나>

1. 중복값 처리 -> 그냥 하나로 처리해도 됨
    교양(0000)에서는 단순 중복 (챕터별 표시 이슈)
    전공에서는 '합반' 키워드 이슈 -> 어차피 합반에 해당하는 모든 단과대의 학정번호가 부여되어 있음
                                 -> 이외에 창업 같은 챕터 중복 이슈 -> 이것도 단순 삭제
    매치업에서는 -> 기초 -> 어차피 모두 교양이고 중복값없음
                 -> 심화 -> 전선으로 처리되는 과(경영, 화학, 컴공)의 학정번호로만 구성되어있음. 
                    -> 따라서, 다른과가 듣는 경우만 이수를 '일선'이라고 하면 될 것 같음.
2. ‘분반_최종’을 ‘수강가능한 학과’로 이름을 바꾸고, 나머지 빈 셀을 모든 학과 코드 적기
3. ‘전공 인정 학과’ 새 칼럼을 생성하고, ‘이수’칼럼이 ‘전선’, ‘전필’인 경우에 해당하는 칼럼으로,
학정번호 앞 4글자가 아래 표에서 찾아서 해당 학과코드를 넣기
4. 강의시간 좌표형 변환
5. 강의 테마 칼럼 추가
'이수'가 교선,교필일 경우에 해당하는 칼럼으로 → 코드대신 엑셀에서 드래그가 훨씬 나을거 같음

8. ‘분반_최종’을 ‘수강가능한 학과’로 이름을 바꾸고, 나머지 빈 셀을 모든 학과 코드 적기

In [28]:
# 칼럼 이름 변경
df = df.rename(columns={'분반_최종': '수강가능한 학과'})

# 비어있는 셀을 특정 학과 코드로 채우기
all_departments = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
df['수강가능한 학과'] = df['수강가능한 학과'].replace('', all_departments)


9. ‘전공 인정 학과’ 새 칼럼을 생성하고, ‘이수’칼럼이 ‘전선’, ‘전필’인 경우에 해당하는 칼럼으로,
학정번호 앞 4글자가 아래 표에서 찾아서 해당 학과코드를 넣기

In [29]:
# 학과 코드 매핑
department_mapping = {
    '7000': 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2',
    '7060': 'E1,E2,E3,E4',
    '7070': 'E5,E6',
    '7420': 'E7,E8',
    '7320': 'J1,J2',
    '7340': 'J3,J4',
    '7460': 'T1,T2',
    'I000': 'C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8',
    'I020': 'C1,C2,C3',
    'I030': 'C4,C5,C6',
    'I040': 'C7,C8,C9',
    'I050': 'J5,J6,J7,J8',
    '1000': 'A2,K1,K2,K3,A1',
    '1170': 'A2',
    '1140': 'K1,K2',
    '1160': 'K3',
    '1270': 'A1',
    '6000': 'N1,N2,N3,N4,N5,P1,test2',
    '6030': 'N1',
    '6100': 'N2,N3',
    '6050': 'N4,N5',
    '6130': 'P1',
    '6120': 'test2',
    '3000': 'R1,R2,M1,M2,R3,R4,R5',
    '3040': 'R1',
    '3220': 'R2',
    '3230': 'M1,M2',
    '3110': 'R3',
    '3210': 'R4,R5',
    'F000': 'S1,S2,L1,L2,L3,S3,test1',
    'F020': 'S1,S2',
    'F030': 'L1,L2,L3',
    'F040': 'S3',
    'F050': 'test1',
    '5000': 'B1,B2,B3,B4,B5,B6',
    '5080': 'B1,B2,B3,B4',
    '5100': 'B5,B6',
    '0000': 'V1,V2,V3,V4', ## 주의!! 
    'J020': 'V1',
    'J030': 'V2',
    'J040': 'V3',
    'J050': 'V4'
}

# '전공 인정 학과' 칼럼 생성
def get_major_recognition(department_code, course_type):
    if course_type in ['전선', '전필']:
        return department_mapping.get(department_code, '')
    return ''

df['전공 인정 학과'] = df.apply(lambda row: get_major_recognition(row['학정번호'][:4], row['이수']), axis=1)


10. 강의시간 좌표형 변환

In [31]:
import pandas as pd
import re

# 요일을 숫자로 매핑
day_mapping = {
    '월': '1',
    '화': '2',
    '수': '3',
    '목': '4',
    '금': '5',
    '토': '6'
}

def transform_class_time(row):
    time_str = row['강의시간 (요일,교시)']
    lecture_type = row['강의유형']
    course_name = row['과목명']

    # lecture_type이 None인지 확인
    if pd.isna(lecture_type):
        lecture_type = ''

    # time_str이 None 또는 NaN인지 확인
    if pd.isna(time_str) or time_str == 'Null':
        # 규칙 1 적용
        if '원격수업100' in lecture_type:
            return 0
        # 규칙 2 적용
        elif 'KW-VIP 2024-2' in course_name or '학과별 운영방식에 따라 별도로 운영' in lecture_type:
            return '학과별 운영방식에 따라 별도로 운영함'
        return time_str

    # time_str이 None이 아니고, 유효한 경우에만 처리
    if isinstance(time_str, str):
        # 정규 표현식을 사용하여 요일과 교시 추출
        pattern = re.compile(r'([월화수목금토])(\d+(?:,\d+)*)')
        matches = pattern.findall(time_str)
        
        transformed = []
        
        for match in matches:
            day = match[0]
            periods = match[1].split(',')  # 여러 교시를 개별적으로 처리
            if day in day_mapping:
                for period in periods:
                    transformed.append(f"({day_mapping[day]}:{period})")
            else:
                print(f"Day not in mapping: {day}")
        
        return ','.join(transformed)
    
    # 기본적으로 None 반환
    return None

# Apply the transformation
df['강의시간 (요일,교시)'] = df.apply(transform_class_time, axis=1)

df


,학정번호,과목명,연계,이수,학점,시수,담당교수,"강의시간 (요일,교시)",강의유형,분반_원본,외국인재학생,국내재학생,체육특기자,재수강용,타학과수강불가,학년,수강가능한 학과,전공 인정 학과
0,0000-1-3093-01,한국영화사로보는가족,NaN,교선,3,3,강성률,"(5:7),(5:8)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
1,0000-1-4428-01,디지털신기술기반신재생에너지,NaN,교선,3,3,손희상,"(2:7),(2:8)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
2,0000-1-4430-01,"슬기로운가정생활,관계놀이",NaN,교선,3,3,최윤희 윤란호,"(4:8),(4:9)","원격수업100,팀티칭강의",,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
3,0000-1-4430-02,"슬기로운가정생활,관계놀이",NaN,교선,3,3,최윤희 윤란호,"(4:8),(4:9)","원격수업101,팀티칭강의,외국인전 용",외국인만 수강가능,True,False,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
4,0000-1-4431-01,ICT가바꾸는미래의법,NaN,교선,3,3,선지원,"(2:5),(2:6)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
5,0000-1-4848-01,현대중국예술과문화코드이해,NaN,교선,3,3,전보옥,"(1:7),(1:8)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
6,0000-1-4849-01,배터리신기술과우리의생활,NaN,교선,3,3,손희상,"(4:7),(4:8)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
7,0000-1-4850-01,수학적사고로보는세상,NaN,교선,3,3,최종성,"(3:7),(3:8)",원격수업100?,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
8,0000-3-9747-01,안전및조직관리사례연구,NaN,무관,3,4,NaN,"(2:1),(2:2),(4:1),(4:2)",NaN,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",
9,0000-4-9748-01,조직리더십사례연구,NaN,무관,3,4,NaN,"(2:1),(2:2),(4:1),(4:2)",NaN,,NaN,NaN,,,,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",


(수동) 강의 테마 칼럼 추가
’이수’가 교선,교필일 경우에 해당하는 칼럼으로 → 코드대신 엑셀에서 드래그가 훨씬 나을거 같음

In [32]:
df.to_csv("./누락된것들.csv", encoding="cp949")

테마

광운인되기

실용영어

융합적사고와글쓰기

정보

과학과기술

글로벌문화와제2외국어

사회와경제

수리와자연(물리)

수리와자연(생물)

수리와자연(수학)

수리와자연(화학)

예술과체육

인간과철학

K-MOOC

서울권역e-러닝

매치업집중이수제

기초수학

외국어로서의한국어

군사학

검증작업 필수!! 누락된 내용이 발생

복잡한 데이터 구조: PDF에는 셀 내부에 줄 바꿈이나 여러 수준의 정보가 포함된 행이 있습니다. 이러한 복잡한 구조를 추출하는 과정에서 일부 데이터가 누락되거나 잘못 처리되었을 수 있습니다.

일관되지 않은 데이터 형식: 일부 행은 열의 개수나 셀이 여러 줄로 나뉘어 있는 등 형식이 다를 수 있으며, 이로 인해 데이터가 누락되거나 정렬되지 않았을 가능성이 있습니다.
